Transform은 dataset을 creat하는 동안
PIL image, tensor, ndarrays or custom data
에 사용될 수 있다.

- Image
    - RandomCrop...등등
- Tensor
    - LinearTransformation, Normalize, RandonErasing
- Conversion
    - ToPILImage
    - ToTensor
- Generic
    - use lambda
- Custom
    - write own class
- compose multiple transform
    - transforms.Compose([~,~])

In [0]:
import torch
import torchvision
from torch.utils.data import Dataset
import numpy as np

In [0]:
class WineDataset(Dataset):
    def __init__(self, transform=None):
        xy = np.loadtxt("https://raw.githubusercontent.com/python-engineer/pytorchTutorial/master/data/wine/wine.csv", 
            delimiter=",", dtype = np.float32, skiprows=1)
        self.n_samples = xy.shape[0]
        self.x = xy[:,1:]   # shape (178, 13) 2D tensor를 만들어줘야한다
        self.y = xy[:,[0]]  # shape (178, 1)
        self.transform = transform

    def __getitem__(self, index):
        sample = self.x[index], self.y[index]
        if self.transform:
            sample = self.transform(sample)
        return sample

    def __len__(self):
        return self.n_samples

class ToTensor() :
    def __call__(self, sample):
        inputs, targets = sample
        return torch.from_numpy(inputs), torch.from_numpy(targets)

class MulTransform() :
    def __init__(self, factor):
        self.factor = factor

    def __call__(self, sample):
        inputs, targets = sample
        inputs *= self.factor
        return inputs, targets

In [0]:
# 1
dataset = WineDataset()
first_data = dataset[0]
features, label = first_data
print(type(features), type(label))

dataset = WineDataset(transform=ToTensor())
first_data = dataset[0]
features, label = first_data
print(type(features), type(label))

<class 'numpy.ndarray'> <class 'numpy.ndarray'>
<class 'torch.Tensor'> <class 'torch.Tensor'>


In [0]:
# 2
dataset = WineDataset()
first_data = dataset[0]
features, label = first_data
print(features)

composed = torchvision.transforms.Compose([ToTensor(),
                                           MulTransform(2)])
dataset = WineDataset(transform=composed)
first_data = dataset[0]
features, label = first_data
print(features)

[1.000e+00 1.423e+01 1.710e+00 2.430e+00 1.560e+01 1.270e+02 2.800e+00
 3.060e+00 2.800e-01 2.290e+00 5.640e+00 1.040e+00 3.920e+00 1.065e+03]
tensor([2.0000e+00, 2.8460e+01, 3.4200e+00, 4.8600e+00, 3.1200e+01, 2.5400e+02,
        5.6000e+00, 6.1200e+00, 5.6000e-01, 4.5800e+00, 1.1280e+01, 2.0800e+00,
        7.8400e+00, 2.1300e+03])
